In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from asset_data_module import read_close_prices_all_merged
from features import make_feature_windows
tf.keras.utils.set_random_seed(42)


In [45]:
markets = ['dow30']
# markets = ['commodities']
# start_date, end_date = "2022-01-01", "2025-11-28"
start_date, end_date = "2024-06-01", "2025-11-28"

_, close_df = read_close_prices_all_merged(markets, after_date=start_date)
close_df = close_df.loc[:end_date]

rolling = make_feature_windows(
    close_prices=close_df,
    lookback=50,
    horizon=1,
    days_per_week=2
)
close_df.shape, len(rolling)

((375, 30), 137)

In [46]:
print(rolling[0]['past_weekly_returns'].shape)
rolling[0]['past_weekly_returns'].head()

(50, 30)


,dow30:AAPL,dow30:AMGN,dow30:AXP,dow30:BA,dow30:CAT,dow30:CRM,dow30:CSCO,dow30:CVX,dow30:DIS,dow30:DOW,...,dow30:MSFT,dow30:NKE,dow30:PFE,dow30:PG,dow30:RTX,dow30:TRV,dow30:UNH,dow30:V,dow30:VZ,dow30:WMT
0,0.009438,-0.000130,-0.009288,0.028205,-0.005781,-0.000380,-0.013597,-0.015876,-0.012435,-0.006800,...,0.025051,-0.000848,0.008155,0.008467,0.009916,-0.009218,0.011354,0.015123,0.008988,0.019111
1,0.005194,-0.007707,-0.008644,0.002052,-0.001549,0.022243,-0.003919,0.007073,0.000394,0.000718,...,-0.000378,0.023368,-0.033377,0.006064,-0.003048,0.021550,-0.025016,0.015077,-0.009965,-0.018200
2,0.050798,-0.014264,-0.035612,-0.025232,-0.004968,-0.003562,-0.001528,0.002623,-0.006620,0.007508,...,0.020619,-0.007068,-0.019432,0.002511,-0.011068,-0.011820,0.011207,-0.014458,-0.012783,0.012820
3,0.033654,-0.007343,-0.010476,-0.026217,-0.002907,-0.050903,-0.005477,-0.023601,-0.007763,-0.001604,...,0.020361,-0.017785,-0.013650,-0.006409,-0.017835,-0.011237,0.002174,-0.012751,-0.015960,-0.000450
4,0.011278,0.015887,0.026996,-0.012866,-0.012208,0.006311,0.003728,0.002287,0.014186,-0.014916,...,0.015259,0.008669,-0.024530,0.006528,-0.005346,0.005106,-0.012127,-0.000074,-0.008077,0.010737


In [47]:
rolling[0]['X_feat'].head()

,mom_1w,mom_4w,mom_12w,vol_1w,vol_4w,sharpe_1w,sharpe_4w,vol_ratio,max_drawdown
dow30:AAPL,-0.815335,0.571021,-0.150967,0.379078,-0.053702,-0.187871,0.585219,0.348134,0.014406
dow30:AMGN,0.252263,-1.075911,-1.667085,0.665689,-0.565586,-0.181922,-1.614424,0.414970,0.563006
dow30:AXP,0.689067,-0.373531,-0.076728,-0.132065,0.554010,-0.179532,-0.252939,-0.191048,0.140703
dow30:BA,-0.411102,1.314572,0.007595,0.317401,0.578818,-0.186059,1.045019,-0.045177,-1.412233
dow30:CAT,-0.114660,-0.956522,0.432995,-0.627754,-0.351098,-0.218349,-1.217257,-0.679821,0.010206


In [48]:
rolling[0]['y_ret'].head(3)

dow30:AAPL    0.003637
dow30:AMGN    0.004078
dow30:AXP    -0.009789
dtype: float64

In [49]:
def panel_from_windows(windows, x_key="past_weekly_returns", y_key="y_ret"):
    X_list, y_list = [], []
    meta_rows = []

    for w_idx, w in enumerate(windows):
        if x_key == 'past_weekly_returns':
            X_df = w[x_key].T          # assets x n_lookback (weeks) (DataFrame)
        elif x_key == 'X_feat':
            X_df = w[x_key]          # assets x n_features (DataFrame)
        y_ser = w[y_key]           # assets (Series or array)

        if not isinstance(y_ser, pd.Series):
            y_ser = pd.Series(y_ser, index=X_df.index)

        assets = X_df.index.intersection(y_ser.index)
        Xw = X_df.loc[assets].to_numpy(dtype=np.float32)
        yw = y_ser.loc[assets].to_numpy(dtype=np.float32)

        mask = np.isfinite(Xw).all(axis=1) & np.isfinite(yw)
        Xw, yw = Xw[mask], yw[mask]
        assets_kept = assets.to_numpy()[mask]

        X_list.append(Xw)
        y_list.append(yw)

        t0, t1 = w.get("t0", None), w.get("t1", None)
        for a in assets_kept:
            meta_rows.append((w_idx, a, t0, t1))

    X = np.vstack(X_list) ## weeks*assets x n_lookback/n_features
    y = np.concatenate(y_list)
    meta = pd.DataFrame(meta_rows, columns=["window_idx", "asset", "t0", "t1"])
    return X, y, meta

In [50]:
X_past_returns, y, meta = panel_from_windows(rolling, x_key='past_weekly_returns')
X, y, meta = panel_from_windows(rolling, x_key='X_feat')
X.shape, y.shape, meta.head() ## len(rolling)*n_asset -- each row is a feature set -- to predict y

((4110, 9),
 (4110,),
    window_idx       asset         t0         t1
 0           0  dow30:AAPL 2024-10-23 2024-10-25
 1           0  dow30:AMGN 2024-10-23 2024-10-25
 2           0   dow30:AXP 2024-10-23 2024-10-25
 3           0    dow30:BA 2024-10-23 2024-10-25
 4           0   dow30:CAT 2024-10-23 2024-10-25)

In [51]:
windows = rolling  # your rolling list

W = meta["window_idx"].nunique()
split_w = int(0.8 * W)

train_mask = (meta["window_idx"] < split_w).values
test_mask  = (meta["window_idx"] >= split_w).values

X_train_raw, y_train = X[train_mask], y[train_mask]
X_test_raw,  y_test  = X[test_mask],  y[test_mask]
X_past_returns_test_raw = X_past_returns[test_mask]

# small validation from the tail of the training windows
val_w = max(int(0.1 * split_w), 1)
val_start = split_w - val_w
val_mask = ((meta["window_idx"] >= val_start) & (meta["window_idx"] < split_w)).values
tr2_mask = (meta["window_idx"] < val_start).values

X_tr_raw, y_tr = X[tr2_mask], y[tr2_mask]
X_va_raw, y_va = X[val_mask], y[val_mask]

scaler = StandardScaler()
X_tr = scaler.fit_transform(X_tr_raw).astype(np.float32)
X_va = scaler.transform(X_va_raw).astype(np.float32)
X_te = scaler.transform(X_test_raw).astype(np.float32)

y_tr = y_tr.astype(np.float32)
y_va = y_va.astype(np.float32)
y_te = y_test.astype(np.float32)

In [52]:
X_tr.shape, X_va.shape, X_te.shape, y_tr.shape, y_va.shape, y_te.shape, f"{y_te.shape[0]/close_df.shape[1]} test periods"

((2970, 9), (300, 9), (840, 9), (2970,), (300,), (840,), '28.0 test periods')

In [53]:
def build_mlp(in_dim, hidden=(64, 32), dropout=0.1, lr=1e-3):
    inputs = keras.Input(shape=(in_dim,))
    x = inputs
    for h in hidden:
        x = layers.Dense(h, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mse",
        metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
    )
    return model

model = build_mlp(in_dim=X_tr.shape[1], hidden=(16, 8), dropout=0.1, lr=3e-4)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=20, restore_best_weights=True
    )
]

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    epochs=150,
    batch_size=256,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.2811 - mae: 0.3932 - val_loss: 0.1627 - val_mae: 0.2833
Epoch 2/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2376 - mae: 0.3566 - val_loss: 0.1364 - val_mae: 0.2561
Epoch 3/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2060 - mae: 0.3266 - val_loss: 0.1166 - val_mae: 0.2350
Epoch 4/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1873 - mae: 0.3098 - val_loss: 0.1013 - val_mae: 0.2176
Epoch 5/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1499 - mae: 0.2833 - val_loss: 0.0889 - val_mae: 0.2037
Epoch 6/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1426 - mae: 0.2677 - val_loss: 0.0790 - val_mae: 0.1927
Epoch 7/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1275 - mae: 0.2546 - val_loss: 0.0707 - val_mae: 0.1832
Epoch 8/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1214 - mae: 0.2447 - val_loss: 0.0640 - val_mae: 0.1752
Epoch 9/150
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.108

In [54]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

def prediction_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    sign_acc = (np.sign(y_true) == np.sign(y_pred)).mean()
    corr = np.corrcoef(y_true, y_pred)[0, 1] if len(y_true) > 1 else np.nan
    
    return {"MSE": round(mse, 5), "MAE": round(mae, 5), "R2": round(r2, 5), "SignAcc": round(float(sign_acc), 5), "Corr": round(float(corr), 5)}


In [55]:
y_pred_sm = X_past_returns_test_raw.astype(np.float32).mean(axis=1)
# X_te_raw = scaler.inverse_transform(X_te)
# y_pred_sm = X_te_raw.mean(axis=1)
print("Sample Mean")
print(prediction_metrics(y_te, y_pred_sm))

y_pred = model.predict(X_te, batch_size=1024).squeeze()
print("MLP")
print(prediction_metrics(y_te, y_pred))

print(y_te.mean())

Sample Mean
{'MSE': 0.00029, 'MAE': 0.01161, 'R2': -0.0116, 'SignAcc': 0.53333, 'Corr': 0.06513}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
MLP
{'MSE': 0.00051, 'MAE': 0.01567, 'R2': -0.75877, 'SignAcc': 0.54405, 'Corr': 0.03679}
0.0013321199


In [56]:
def baseline_sample_mean(test_window):
    # past_weekly_returns: (lookback periods) x (assets)
    return test_window["past_weekly_returns"].mean(axis=0)  # pd.Series indexed by asset

Walk-Forward Evaluation

In [57]:
def walk_forward_eval_mlp(
    rolling,
    train_len=150,          # number of windows to train on each step
    x_key="X_feat",
    y_key="y_ret",
    hidden=(64, 32),
    dropout=0.1,
    lr=3e-4,
    epochs=150,
    batch_size=256,
    seed=42
):
    tf.keras.utils.set_random_seed(seed)

    all_rows = []
    week_metrics = []

    for i in range(train_len, len(rolling)):
        train_windows = rolling[i-train_len:i]
        test_window   = rolling[i]

        # --- build train panel ---
        X_train, y_train, meta = panel_from_windows(train_windows, x_key=x_key, y_key=y_key)
        if X_train.shape[0] == 0:
            continue

        # --- scaler on TRAIN only ---
        scaler = StandardScaler()
        X_train_sc = scaler.fit_transform(X_train).astype(np.float32)

        # --- build test cross-section ---
        X_test_df = test_window[x_key]            # assets x features
        y_test = test_window[y_key]
        if not isinstance(y_test, pd.Series):
            y_test = pd.Series(y_test, index=X_test_df.index)

        assets = X_test_df.index.intersection(y_test.index).sort_values()
        X_test = X_test_df.loc[assets].to_numpy(np.float32)
        y_true = y_test.loc[assets].to_numpy(np.float32)

        mask = np.isfinite(X_test).all(axis=1) & np.isfinite(y_true)
        assets = assets[mask]
        X_test = X_test[mask]
        y_true = y_true[mask]

        X_test_sc = scaler.transform(X_test).astype(np.float32)

        # --- train model (fresh each step) ---
        model = build_mlp(in_dim=X_train_sc.shape[1], hidden=hidden, dropout=dropout, lr=lr)
        es = keras.callbacks.EarlyStopping(monitor="loss", patience=15, restore_best_weights=True)

        model.fit(
            X_train_sc, y_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[es]
        )

        # --- predict ---
        y_pred = model.predict(X_test_sc, batch_size=1024, verbose=0).squeeze()

        # --- baseline: sample mean of past period returns ---
        y_pred_sm_ser = baseline_sample_mean(test_window).loc[assets]
        y_pred_sm = y_pred_sm_ser.to_numpy(np.float32)

        # store per-asset predictions
        for a, yt, yp, ypsm in zip(assets, y_true, y_pred, y_pred_sm):
            all_rows.append(
                {"window_idx": i, "asset": a, "y": float(yt), "pred_mlp": float(yp), "pred_sm": float(ypsm)}
            )

        # per-window metrics (cross-section)
        m_mlp = prediction_metrics(y_true, y_pred)
        m_sm  = prediction_metrics(y_true, y_pred_sm)
        week_metrics.append({"window_idx": i, **{f"mlp_{k}": v for k,v in m_mlp.items()},
                                **{f"sm_{k}": v for k,v in m_sm.items()}})

    preds_df = pd.DataFrame(all_rows)
    week_df  = pd.DataFrame(week_metrics)

    # pooled metrics over all (window, asset) test points
    pooled_mlp = prediction_metrics(preds_df["y"], preds_df["pred_mlp"])
    pooled_sm  = prediction_metrics(preds_df["y"], preds_df["pred_sm"])

    return preds_df, week_df, pooled_mlp, pooled_sm

In [58]:
preds_df, week_df, pooled_mlp, pooled_sm = walk_forward_eval_mlp(
    rolling=rolling,
    train_len=100,   # e.g., last 52 periods (with days_per_week=2 that's ~104 trading days)
    x_key="X_feat",
    y_key="y_ret",
    hidden=(16,8)
)

print("Pooled MLP:", pooled_mlp)
print("Pooled  SM:", pooled_sm)
print("Weekly win-rate (MAE):", (week_df["mlp_MAE"] < week_df["sm_MAE"]).mean())

Pooled MLP: {'MSE': 0.00097, 'MAE': 0.02018, 'R2': -2.47369, 'SignAcc': 0.4955, 'Corr': 0.03159}
Pooled  SM: {'MSE': 0.0003, 'MAE': 0.01162, 'R2': -0.06393, 'SignAcc': 0.4982, 'Corr': -0.05135}
Weekly win-rate (MAE): 0.13513513513513514
